In [1]:
import numpy as np
import pandas as pd
from dask import dataframe as dd
from dask.distributed import Client

/Users/NothingToLose/clepy/myvenv/lib/python3.8/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:63238,Cluster Workers: 4 Cores: 4 Memory: 4.29 GB


In [ ]:
%%time
df = pd.read_csv('sample_data.csv')

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
%%time
ddf = dd.read_csv('sample_data.csv')

In [ ]:
ddf.head()